In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,7,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,222,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,223,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,583,"CREATE TABLE accounts (\tid integer,\tname bpc..."


Task 1:

# Creating a Partitioned Running Total Using Window Functions

Now, modify your query from the previous quiz to include partitions. Still create a running total of standard_amt_usd (in the orders table) over order time, but this time, date truncate occurred_at by year and partition by that same year-truncated occurred_at variable. Your final table should have three columns: One with the amount being added for each row, one for the truncated date, and a final column with the running total within each year.

In [15]:
# sqlite version

pd.read_sql_query(sql='''
SELECT standard_amt_usd, strftime('%Y', occurred_at) year, SUM(standard_amt_usd) OVER (PARTITION BY strftime('%Y', occurred_at) ORDER BY occurred_at) running_total
FROM  orders;
''', con=conn)[:103]

,standard_amt_usd,year,running_total
0,0.00,2013,0.00
1,2445.10,2013,2445.10
2,2634.72,2013,5079.82
3,0.00,2013,5079.82
4,2455.08,2013,7534.90
...,...,...,...
98,2420.15,2013,132504.46
99,2569.85,2014,2569.85
100,0.00,2014,2569.85
101,184.63,2014,2754.48


Solution:

SELECT standard_amt_usd,
       DATE_TRUNC('year', occurred_at) as year,
       SUM(standard_amt_usd) OVER (PARTITION BY DATE_TRUNC('year', occurred_at) ORDER BY occurred_at) AS running_total
FROM orders

If you'd like another example of partitioning, check out the top answer from this Stack Overflow post: “Partition By” Keyword
https://stackoverflow.com/questions/561836/oracle-partition-by-keyword